In [1]:
import numpy as np
from pprint import pprint as pp
import re
import math as maths
from scipy.optimize import minimize
from scipy.optimize import Bounds
from ase.units import kB

pattern1 = re.compile("\n\n\n")
pattern2 = re.compile("\n\n")

In [2]:
# Read clusters.out
clusters = {}


with open('clusters.out','r') as fclusters:
    temp_clusters = fclusters.read().split('\n\n')

for idx, cluster in enumerate(temp_clusters):
    if cluster == '':
        continue
    line = cluster.split('\n')
    multiplicity = int(line[0])
    length = float(line[1])
    num_points = int(line[2])
    clusters[idx] = {'mult':multiplicity, 'length':length, 'type':num_points}
    
num_clusters = len(clusters)

In [3]:
# Read config.out
configs = {}

fconfig = open('config.out','r')
_ = next(fconfig)
temp_config = fconfig.read()#.split('\n\n')
temp_config = pattern1.split(temp_config)

for idx, config in enumerate(temp_config):
    if config == '':
        continue
    num_points = int(config[0])
    config = pattern2.split(config[2:])
    min_coords = []
    for _ in range(num_points):
        min_coords.append(config[_].split('\n')[0])
    configs[idx] = {'subclus': list(map(int,min_coords)), 'num_of_subclus': len(min_coords)}

In [4]:
# Read kb.out

kb = {}

fkb = open('kb.out','r')
_ = next(fkb)

temp_kb = fkb.read()
temp_kb = temp_kb.split('\n')

for idx, kbcoeff in enumerate(temp_kb):
    if kbcoeff == '':
        continue
    kb[idx] = float(kbcoeff)

fkb.close()

In [5]:
# Read configcoeff.out

configcoef = {}

with open('configcoef.out','r') as fsubmult:
    _ = next(fsubmult)
    temp_submult = fsubmult.read()
    temp_submult = pattern2.split(temp_submult)
    
for idx, submult in enumerate(temp_submult):
    submult = submult.split('\n')
    while("" in submult) :
        submult.remove("")
    configcoef[idx] = list(map(float,submult[1:]))

In [6]:
# Read vmat.out

vmat = {}

with open('vmat.out') as fvmat:
    _ = next(fvmat)
    temp_vmat = fvmat.read()
    #temp_vmat = temp_vmat.split('\n')
    temp_vmat = pattern2.split(temp_vmat)
    
    while("" in temp_vmat) :
        temp_vmat.remove("")
    
    for clus_idx, mat in enumerate(temp_vmat):
        mat = mat.split('\n')
        mat_float = np.empty(list(map(int, mat[0].split(' '))))
        for idx, row in enumerate(mat[1:]):
            mat_float[idx] = list(map(float,row.split(' ')[:-1]))
        
        vmat[clus_idx] = mat_float

In [7]:
# Read eci
eci = {}

with open('eci.out') as feci:
    _ = next(feci)
    temp_eci = feci.read()
    temp_eci = temp_eci.split('\n')

for idx, eci_val in enumerate(temp_eci):
    if eci_val == '':
        continue
    eci[idx] = float(eci_val)

eci

{0: 0.006143, 1: -0.018951, 2: 0.01147, 3: -0.009881, 4: 2.3e-05, 5: 0.002117}

In [ ]:
corrs = np.array([1.0,-0.14286,0.02041,0.02041,0.00292,0.0])

## Doing the Optimisation

In [44]:
#corrs = np.array([1.0,-1,0.02041,0.02041,0.00292,0.0])

#cluster_optim = []
#for cluster_idx, cluster in clusters.items():
#    cluster_optim.append(cluster['type'] >= 2)
    
#corrs_optim = corrs[cluster_optim]
#corrs_fixed = corrs[[not elem for elem in cluster_optim]]
FIXED_CORR_1 = 1.0
corrs = np.array([1.0,FIXED_CORR_1,0.02041,0.02041,0.00292,0.0])
#corrs = np.array([1.0,FIXED_CORR_1,1.00000, 1.00000, -1.00000, 1.00000])

bounds_corrs = Bounds(np.array([-1]*len(corrs)),np.array([1]*len(corrs)))

def constraint_rhos_sum(corrs, vmat, clusters, configcoef,):
    
    rho_sum = []
    
    #corrs = np.concatenate((corrs_fixed,corrs_optim))
    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)
        return rho
    
    for cluster_idx, _ in clusters.items():
        rho = clus_prob(cluster_idx)
        rho_sum.append(np.sum(configcoef[cluster_idx]*rho))
    
    return np.sum(1 - np.array(rho_sum))

def constraint_rhos_single(corrs, vmat, clusters, configcoef,):
    
    all_pos = []
    
    #corrs = np.concatenate((corrs_fixed,corrs_optim))
    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)
        return rho
    
    for cluster_idx, _ in clusters.items():
        rho = clus_prob(cluster_idx)
        all_pos.append(((rho >= 0).all() and (rho <= 1).all()))
    
    return 1 - int(all(all_pos))

def constraint_singlet(corrs):
    
    return corrs[1] - FIXED_CORR_1

def constraint_zero(corrs):
    
    return corrs[0] - 1.0
                    
       
def F(corrs, vmat, kb, clusters, configs, configcoef,):

    S = 0
    H = 0
    #corrs = np.concatenate((corrs_fixed,corrs_optim))
    
    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)

        return rho
    
    def inner_sum(cluster_idx):
        isum = 0
        rho = clus_prob(cluster_idx)
        #print(rho)
        try:
            for i in range(configs[cluster_idx]['num_of_subclus']):
                isum += configcoef[cluster_idx][i] * rho[i] * maths.log(rho[i])
        except ValueError as ve:
            #print(ve)
            pass
        #for i in range(configs[cluster_idx]['num_of_subclus']):
        #        isum += configcoef[cluster_idx][i] * rho[i] * np.log(rho[i])
        return isum 
            
    for cluster_idx, cluster in clusters.items():
        H += cluster['mult']*eci[cluster_idx]*corrs[cluster_idx]
        S += kb[cluster_idx]*inner_sum(cluster_idx)

    T = 3000
    
    #print(corrs)
    return S
    #return H - T*kB*S

In [45]:
res = minimize(F, 
               corrs, 
               args=(vmat, kb, clusters, configs, configcoef,),
               method='trust-constr', 
               options={'verbose': 1,},#'gtol': 1e-3, 'maxiter': 200},
               jac='3-point',
               constraints=[{'fun': constraint_rhos_sum, 'type': 'eq', 'args': (vmat, clusters, configcoef,)},
                            {'fun': constraint_rhos_single, 'type': 'eq', 'args': (vmat, clusters, configcoef,)},
                            {'fun': constraint_singlet, 'type': 'eq'},
                            {'fun': constraint_zero, 'type':'eq'},
                           ],
               bounds=bounds_corrs
              )
#

`xtol` termination condition is satisfied.
Number of iterations: 268, function evaluations: 3484, CG iterations: 262, optimality: 8.10e-14, constraint violation: 1.00e+00, execution time:  0.8 s.


In [46]:
for cluster_idx in clusters:
    print(np.matmul(vmat[cluster_idx],res.x))

print('=====================')
print(res.x)

[1.]
[-4.87127005e-12  1.00000000e+00]
[-0.25001112  0.25001112  0.74998888]
[-0.24998664  0.24998664  0.75001336]
[-2.49998934e-01 -1.21882067e-05  2.50023311e-01  1.22983107e-05
  2.49998824e-01  4.99990053e-01]
[-0.18748972 -0.06250921  0.06252151  0.06249702  0.1875018   0.31248825]
[ 1.00000000e+00  1.00000000e+00 -4.44901478e-05  5.34559220e-05
 -4.40497221e-05  5.92571983e-05]


In [47]:
res

 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 262
      cg_stop_cond: 4
            constr: [array([1.95881134e-09]), array([1]), array([-3.16725979e-10]), array([-3.26468519e-10]), array([ 1.00000000e+00,  1.00000000e+00, -4.44901478e-05,  5.34559220e-05,
       -4.40497221e-05,  5.92571983e-05])]
       constr_nfev: [1876, 1876, 1876, 1876, 0]
       constr_nhev: [0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0]
    constr_penalty: 72497.86383675066
  constr_violation: 1.0
    execution_time: 0.8042042255401611
               fun: 0.0
              grad: array([ 2.92694604, -3.42694604,  0.        ,  0.        ,  0.        ,
        0.        ])
               jac: [array([[-6.00000001,  0.        , -0.        ,  0.        , -0.        ,
         0.        ]]), array([[ 0.,  0., -0.,  0., -0.,  0.]]), array([[ 0.,  1., -0.,  0., -0.,  0.]]), array([[ 1.,  0., -0.,  0., -0.,  0.]]), array([[1., 0., 0., 0., 0., 0.],
       [0.